In [1]:
import os
import numpy
from datetime import datetime, timedelta

In [2]:
matched_record_npy = '/disk1/workspace/20240114/MXD/MXD03_RAY-matched_infos_10km_land.npy'
cloudmask_folder = '/disk4/AHI_CloudMask'

In [3]:
matched_infos = numpy.load(matched_record_npy)

In [4]:
matched_infos[0]

array(['98.355', '15.755', '201810300335', '52.38', '51.409', '96.94',
       '106.621', '37.38', '140.88'], dtype='<U12')

In [5]:
matched_infos[matched_infos[:, 0].astype(float) < 0]

array([], shape=(0, 9), dtype='<U12')

In [6]:
def is_clearsky_record(matched_info_item):
    lon = float(matched_info_item[0][:-1])
    lat = float(matched_info_item[1][:-1])
    mxd_date = datetime.strptime(matched_info_item[2], "%Y%m%d%H%M")
    ahi_date = mxd_date
    if matched_info_item[2][-1:] =='5':
        ahi_date = mxd_date + timedelta(minutes=5)
    ahi_time_str = ahi_date.strftime("%Y%m%d%H%M")
    if ahi_time_str[-4:] == '0240':
        ahi_date_temp = ahi_date - timedelta(minutes=10)
        ahi_time_str = ahi_date_temp.strftime("%Y%m%d%H%M")

    ahi_cm_dat = os.path.join(cloudmask_folder, ahi_time_str[:6], 'AHIcm.v0.'+ahi_time_str+'.dat')
    cloudmask = None
    cloudmask_res = 0.02
    if not os.path.exists(ahi_cm_dat):
        print('no file:', ahi_cm_dat)
        cloudmask = numpy.zeros((6000, 6000))
    else:
        try:
            cloudmask = numpy.fromfile(ahi_cm_dat, dtype=numpy.float32)
            cloudmask[cloudmask < 0.95] = 0.   # cloud
            cloudmask[cloudmask >= 0.95] = 1.
            cloudmask = cloudmask.reshape(6000, 6000)   # 0.02°
        except Exception as e:
            print(e)
            cloudmask = numpy.zeros((6000, 6000))

    cm_lon_idx = int((lon-85.)/cloudmask_res)
    cm_lat_idx = int((60.-lat)/cloudmask_res)
    is_clearsky = int(cloudmask[cm_lat_idx][cm_lon_idx])
    
    return is_clearsky


def clear_sky_count(matched_info):
    all_record = numpy.zeros((12,))
    clear_record = numpy.zeros((12,))
    for pt_item in matched_info:
        pt_lat = float(pt_item[1])
        if pt_lat <= 60 and pt_lat > 50:    # 60N-50N
            all_record[0] = all_record[0] + 1
            clear_record[0] = clear_record[0] + is_clearsky_record(pt_item)
        elif pt_lat <= 50 and pt_lat > 40:  # 50N-40N
            all_record[1] = all_record[1] + 1
            clear_record[1] = clear_record[1] + is_clearsky_record(pt_item)
        elif pt_lat <= 40 and pt_lat > 30:  # 40N-30N
            all_record[2] = all_record[2] + 1
            clear_record[2] = clear_record[2] + is_clearsky_record(pt_item)
        elif pt_lat <= 30 and pt_lat > 20:  # 30N-20N
            all_record[3] = all_record[3] + 1
            clear_record[3] = clear_record[3] + is_clearsky_record(pt_item)
        elif pt_lat <= 20 and pt_lat > 10:  # 20N-10N
            all_record[4] = all_record[4] + 1
            clear_record[4] = clear_record[4] + is_clearsky_record(pt_item)
        elif pt_lat <= 10 and pt_lat > 0:  # 10N-0
            all_record[5] = all_record[5] + 1
            clear_record[5] = clear_record[5] + is_clearsky_record(pt_item)
        elif pt_lat <= 0 and pt_lat > -10:    # 0-10S
            all_record[6] = all_record[6] + 1
            clear_record[6] = clear_record[6] + is_clearsky_record(pt_item)
        elif pt_lat <= -10 and pt_lat > -20:    # 10S-20S
            all_record[7] = all_record[7] + 1
            clear_record[7] = clear_record[7] + is_clearsky_record(pt_item)
        elif pt_lat <= -20 and pt_lat > -30:  # 20S-30S
            all_record[8] = all_record[8] + 1
            clear_record[8] = clear_record[8] + is_clearsky_record(pt_item)
        elif pt_lat <= -30 and pt_lat > -40:  # 30S-40S
            all_record[9] = all_record[9] + 1
            clear_record[9] = clear_record[9] + is_clearsky_record(pt_item)
        elif pt_lat <= -40 and pt_lat > -50:  # 40S-50S
            all_record[10] = all_record[10] + 1
            clear_record[10] = clear_record[10] + is_clearsky_record(pt_item)
        elif pt_lat <= -50 and pt_lat > -60:  # 50S-60S
            all_record[11] = all_record[11] + 1
            clear_record[11] = clear_record[11] + is_clearsky_record(pt_item)
    return all_record, clear_record

In [7]:
all_item_count, clear_sky_item_count = clear_sky_count(matched_infos)

no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801022220.dat
no file: /disk4/AHI_CloudMask/201809/AHIcm.v0.201809232310.dat
no file: /disk4/AHI_CloudMask/201804/AHIcm.v0.201804072

no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805262220.dat
no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810312

no file: /disk4/AHI_CloudMask/201804/AHIcm.v0.201804092300.dat
no file: /disk4/AHI_CloudMask/201802/AHIcm.v0.201802242240.dat
no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810182300.dat
no file: /disk4/AHI_CloudMask/201804/AHIcm.v0.201804022300.dat
no file: /disk4/AHI_CloudMask/201803/AHIcm.v0.201803062310.dat
no file: /disk4/AHI_CloudMask/201801/AHIcm.v0.201801072240.dat
no file: /disk4/AHI_CloudMask/201805/AHIcm.v0.201805112300.dat
no file: /disk4/AHI_CloudMask/201808/AHIcm.v0.201808232220.dat
no file: /disk4/AHI_CloudMask/201808/AHIcm.v0.201808072220.dat
no file: /disk4/AHI_CloudMask/201806/AHIcm.v0.201806272220.dat
no file: /disk4/AHI_CloudMask/201806/AHIcm.v0.201806272220.dat
no file: /disk4/AHI_CloudMask/201806/AHIcm.v0.201806272220.dat
no file: /disk4/AHI_CloudMask/201806/AHIcm.v0.201806272220.dat
no file: /disk4/AHI_CloudMask/201806/AHIcm.v0.201806272220.dat
no file: /disk4/AHI_CloudMask/201806/AHIcm.v0.201806272220.dat
no file: /disk4/AHI_CloudMask/201806/AHIcm.v0.201806272

In [10]:
latitude_ranges = ['60N-50N', '50N-40N', '40N-30N', '30N-20N', '20N-10N', '10N-0', '0-10S', '10S-20S', '20S-30S', '30S-40S', '40S-50S', '50S-60S']

print('latitudes | all count | clear-sky count | % of clear-sky')
for latitude_range_idx in range(len(latitude_ranges)):
    if all_item_count[latitude_range_idx] > 0:
        print(latitude_ranges[latitude_range_idx], all_item_count[latitude_range_idx], clear_sky_item_count[latitude_range_idx], '('+str(round(100*(clear_sky_item_count[latitude_range_idx]/all_item_count[latitude_range_idx]), 2))+'%)')
    else:
        print(latitude_ranges[latitude_range_idx], all_item_count[latitude_range_idx], clear_sky_item_count[latitude_range_idx], '--')
print('')

latitudes | all count | clear-sky count | % of clear-sky
60N-50N 0.0 0.0 --
50N-40N 0.0 0.0 --
40N-30N 0.0 0.0 --
30N-20N 1903.0 1184.0 (62.22%)
20N-10N 12235.0 4923.0 (40.24%)
10N-0 32836.0 7319.0 (22.29%)
0-10S 31540.0 6837.0 (21.68%)
10S-20S 218.0 0.0 (0.0%)
20S-30S 0.0 0.0 --
30S-40S 0.0 0.0 --
40S-50S 0.0 0.0 --
50S-60S 0.0 0.0 --

